In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import time

import ibdloader
import baseheuristic as bh

rng = np.random.default_rng(2023)

datapath = "../datasets-genotek/"

In [ ]:
#dataset 1: 
#only pure population samples and total ibd length and count between them

#CR_graph_rel.csv
#node_id1,node_id2,label_id1,label_id2,ibd_sum,ibd_n
#node_0,node_5,мордвины,мордвины,29.8172,4

#NC_graph_rel.csv 
#node_id1,node_id2,label_id1,label_id2,ibd_sum,ibd_n
#node_0,node_1,чеченцы,чеченцы,9.76537,1
#node_1,node_138,чеченцы,"кабардинцы,черкесы,адыгейцы",8.01344,1

dataset1fname = datapath+"CR_graph_rel.csv"
df = pd.read_csv(dataset1fname)
print(df)

pairs, weights, labels, labeldict, idxtranslator =\
ibdloader.load_pure( dataset1fname )



#data struсture conventions:
#node indices in datasets are not necessarily starting from 0 and consecutive 
#so they are enumerated in idxtranslator.
#for every idx the label of node with index idxtranslator[idx] is stored in labels[idx]
#length of both arrays labels and idxtranslator coincides with count of available nodes in the dataset  

#in graphs we store consecutive labels starting from 0
#it can be even subset of idxtranslator, so every graph must have its own gr_idxtranslator
#and possibly gr_labels

#example
#idxtranslator = [0 1 3 5 10]
#       labels = [2 0 2 0 2]
# pairs = [0 10 100]
#         [1 5 100]
#         [3 10 100]
#subset after removing node_3
#idxtranslator = [0 1 5 10]
#       labels = [2 0 0 2]

#lets not produce translated pairs
#just graphs with corresponding edges and nodes from 0 and consecutive


# 1. построим распределение суммарных весов и количества ibd-сегментов

In [ ]:
bh.plot_distribution(weights, 50, "Distribution of ibd sum")
bh.plot_distribution(pairs[:,2], 10, "Distribution of ibd count")

# 2. Распределение весов внутри и между классами

In [ ]:
#change plot_distr to true to plot distributions
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
graphdata = bh.composegraphs(pairs, weights, labels, labeldict, idxtranslator)
ncls = graphdata[0]['nodeclasses']
grph = graphdata[0]['graph']
trns = graphdata[0]['translation']
weimatrix, countmatrix, distrs = bh.getweightandcountmatrices(grph, ncls, labeldict, plot_distr=False)

In [ ]:
for dst in distrs:
    print(dst)

In [ ]:
#Optional: fit distributions to popular pdf curves
#distributions here are of ibd sum from individual of one class to the whole another class, so they are
#not symmetric, e.g. distribution from mordvins to belarussians is not the same as distribution from belarussians to mordvins

#it is isteresting to see symmetric distribution as well (distribution on ibd-sum of one link between two classes)
#or distribution of link count from individual of one population to the whole another population (also non-symmetrical)

#total distribution of link count between two individuals is not informative (almost constant 1)


#dist_names = ['alpha', 'gamma', 'norm']
dist_names = ['gamma', 'norm']
for label in distrs:
    #label = "южные-русские"
    data = distrs[label]["data"]
    threshold = distrs[label]["threshold"]
    if threshold is None:
        threshold = 1000
    bins = distrs[label]["bins"]
    bh.plot_and_fit_distribution(data, threshold, bins, f"Distribution of ibd sum for {label}", dist_names)

# 3. Матрица сумм весов внутри и между классами

In [ ]:
#check total ibd sum between classes
for label in ncls:
    print (f"{label}: {ncls[label].shape[0]}")
    
# в тысячах
cm_display = ConfusionMatrixDisplay(weimatrix/1000, display_labels=labeldict.keys()).plot()
cm_display.ax_.set_title("Веса/1000")
plt.show()
cm_display = ConfusionMatrixDisplay(countmatrix, display_labels=labeldict.keys()).plot()
cm_display.ax_.set_title("Количество ребер")
plt.show()

# Классификация

Построим матрицы ошибок по:

1. количеству ребер до известных классов
2. числу ребер на количество членов класса
3. количеству ibd-сегментов до известных классов
4. самому длинный сегмент
5. весу на ребро
6. суммарному весу ребер до известных классов



In [ ]:
start = time.time()
featuredict = bh.getfeatures(grph, grph.nodes, ncls, labeldict, pairs, trns )
print(f"features collected in {time.time()-start} seconds\n")

In [ ]:
simplepredictions = bh.getsimplepred(featuredict)
for feature in featuredict:
    prediction = simplepredictions[feature]
    title = featuredict[feature]["comment"]
    bh.show_prediction(labels, prediction, labeldict.keys(), title)

# Наиболее уверенный классификатор

In [ ]:
#наиболее уверенный классификатор среди нескольких с весами
#nodecount = labels.shape[0]
#labelcount = len(labeldict)


featureweights = {
    "SegmentCount": 0,
    "SegmentCountPerClassize": 1.38,
    "SegmentCountWMult": 0,
    "LongestIbd": 1,
    "IbdSum": 0.41,
    "IbdSumPerEdge": 0
}

#this is useful for the case where no second-best class is available, so several classifiers have infinite confidence
featurepriority = [
    "LongestIbd",
    "SegmentCountPerClassize",
    "IbdSum",
    "SegmentCountWMult",
    "SegmentCount",    
    "IbdSumPerEdge"]

mostconfident = bh.get_most_confident_prediction(featuredict, featureweights, featurepriority)
bh.show_prediction(labels, mostconfident, labeldict.keys(), "Классификация наиболее уверенным классификатором.")

# Простой перебор для поиска наилучшей комбинации весов

In [ ]:
#search for the best parameter combination
#may take hours
featureweightrange = {
    "SegmentCount": [0],
    "SegmentCountPerClassize": np.arange(0.5,1.5,0.1),
    "SegmentCountWMult": [0],    
    "LongestIbd": [1],
    "IbdSum": np.arange(0.1,1.5,0.1),    
    "IbdSumPerEdge": [0]
}

bestweights, bestaccuracy = bh.search_best_weights(featuredict, featureweightrange, featurepriority, labels, trainnodes = None, show_intermediate = False)
print(bestweights, bestaccuracy)
mostconfident = bh.get_most_confident_prediction(featuredict, bestweights, featurepriority)
bh.show_prediction(labels, mostconfident, labeldict.keys(), "Классификация наиболее уверенным классификатором.")

# Разбиение на "тренировочную", "валидационную" и "тестовую" части

Разделим каждый класс train:val:test = 60:20:20, веса выберем по "тренировочной"+"валидационной" выборке, метрики посчитаем по "тестовой" 

In [ ]:
valshare = 0.2
testshare = 0.2
#print(ncls)
#remember that ncls are starting from 0 and consecutive
#also somewhere we should check connectivity, e.g. if test node have connections with train tree at all

permt = bh.getrandompermutation(ncls, rng)
#print(permt)
trainnodeclasses, valnodeclasses, testnodeclasses = bh.dividetrainvaltest(ncls, valshare, testshare, permt)
#here subdivision does not change translation array as we still have the same graph,
#but will compute features only for test nodes and based on links to trainnodeclasses only


print("train|val|test distribution")
for c in trainnodeclasses:
    print(c, ":", trainnodeclasses[c].shape[0], ":", valnodeclasses[c].shape[0], ":", testnodeclasses[c].shape[0])

trainnodes, valnodes, testnodes = bh.gettrainvaltestnodes(trainnodeclasses, valnodeclasses, testnodeclasses)
print("train nodes:", trainnodes.shape[0])
print("val nodes:", valnodes.shape[0])
print("test nodes:", testnodes.shape[0])

testlabels = labels[testnodes]

start = time.time()
featuredict = bh.getfeatures(grph, testnodes, trainnodeclasses, labeldict, pairs, trns )
print(f"features collected in {time.time()-start} seconds\n")

simplepredictions = bh.getsimplepred(featuredict)
for feature in featuredict:
    prediction = simplepredictions[feature]
    #predictedlabels = np.array([ prediction[node] for node in testnodes])
    predictedlabels = prediction[testnodes]
    title = featuredict[feature]["comment"]
    bh.show_prediction(testlabels, predictedlabels, labeldict.keys(), title)


# Наиболее уверенный классификатор для 30%

In [ ]:
featureweights = {
    'IbdSumPerEdge': 0, 
    'IbdSum': 0.5, 
    'LongestIbd': 1, 
    'SegmentCountWMult': 0, 
    'SegmentCountPerClassize': 1.3, 
    'SegmentCount': 0}

#this is useful for the case where no second-best class is available, so several classifiers have infinite confidence
featurepriority = [
    "LongestIbd",
    "SegmentCountPerClassize",
    "IbdSum",
    "SegmentCountWMult",
    "SegmentCount",    
    "IbdSumPerEdge"]

mostconfident = bh.get_most_confident_prediction(featuredict, featureweights, featurepriority)
predictedlabels = mostconfident[testnodes]
bh.show_prediction(testlabels, predictedlabels, labeldict.keys(), "Классификация наиболее уверенным классификатором.")



In [ ]:
#search for the best parameter combination
#may take hours
featureweightrange = {
    "SegmentCount": [0],
    "SegmentCountPerClassize": np.arange(0.5,1.5,0.1),
    "SegmentCountWMult": [0],    
    "LongestIbd": [1],
    "IbdSum": np.arange(0.1,1.5,0.1),    
    "IbdSumPerEdge": [0]
}

#featureweightrange = {
#    "SegmentCount": [0],
#    "SegmentCountPerClassize": [1.38],
#    "SegmentCountWMult": [0],
#    "LongestIbd": [1],
#    "IbdSum": [0.41],
#    "IbdSumPerEdge": [0]
#}


#we will search for best weights on train nodes and then apply most confident classifier with these weights to test nodes
trainlabels = labels[trainnodes] 

bestweights, bestaccuracy = bh.search_best_weights(featuredict, featureweightrange, featurepriority, trainlabels, trainnodes, False)
print(bestweights, bestaccuracy)

mostconfident = bh.get_most_confident_prediction(featuredict, bestweights, featurepriority)
bh.show_prediction(testlabels, mostconfident[testnodes], labeldict.keys(), "Классификация наиболее уверенным классификатором.")

In [ ]:
#repeat for new subdivision
itercount = 50
totalaccr = 0
for itr in range(itercount):
    permt = bh.getrandompermutation(ncls, rng)
    #print(permt)
    trainnodeclasses, testnodeclasses = bh.dividetraintest(ncls, testshare, permt)
    #here subdivision does not change translation array as we still have the same graph,
    #but will compute features only for test nodes and based on links to trainnodeclasses only

    #when it will be neccessary to 
    #print("train|test distribution")
    #for c in trainnodeclasses:
    #    print(c, ":", trainnodeclasses[c].shape[0], ":", testnodeclasses[c].shape[0])

    trainnodes, testnodes = bh.gettraintestnodes(trainnodeclasses, testnodeclasses)
    #print("train nodes:", trainnodes.shape[0])
    #print("test nodes:", testnodes.shape[0])


    testlabels = labels[testnodes]
    trainlabels = labels[trainnodes] 

    start = time.time()
    featuredict = bh.getfeatures(grph, testnodes, trainnodeclasses, labeldict, pairs, trns )
    #print(f"features collected in {time.time()-start} seconds\n")

    bestweights, bestaccuracy = bh.search_best_weights(featuredict, featureweightrange, featurepriority, trainlabels, trainnodes, False)
    print("iter", itr)
    print(bestweights, bestaccuracy)
    mostconfident = bh.get_most_confident_prediction(featuredict, bestweights, featurepriority)
    accr = np.sum(testlabels == mostconfident[testnodes])/testnodes.shape[0]
    totalaccr += accr
    print(f" Accuracy: {accr:.4f}, correct: {np.sum(labels == prediction)}, total: {labels.shape[0]}")
    
    #bh.show_prediction(testlabels, mostconfident[testnodes], labeldict.keys(), "Классификация наиболее уверенным классификатором.")

print("average accuracy:", totalaccr/itercount)